In [9]:
import os
import cv2
import json
import glob
import numpy as np
from tqdm import tqdm
import sys
import pandas as pd

In [4]:
sys.path.append('..')

In [5]:
from config import SCENES_PATH, VIDEOS_PATH, IMPROVED_KEYFRAME_PATH, IMPROVED_MAP_KEYFRAMES_PATH

In [6]:
scenes_dir = "../" + SCENES_PATH
video_dir = "../" + VIDEOS_PATH
keyframes_dir = "../" + IMPROVED_KEYFRAME_PATH
map_keyframes_dir = "../" + IMPROVED_MAP_KEYFRAMES_PATH

In [12]:
def get_frame_indices(scenes):
    frame_idx_list = []
    for start, end in scenes:
        # mid1 = start + (end - start) // 3
        # mid2 = start + 2 * (end - start) // 3
        frame_idx_list.extend([start, end])
    return sorted(set(frame_idx_list))

In [8]:
def write_and_map_keyframes(video_path, frame_idx_list, output_folder):
    video_capture = cv2.VideoCapture(video_path)
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    frames_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    
    keyframes = []
    os.makedirs(output_folder, exist_ok=True)
    
    zfill_size = len(str(len(frame_idx_list)))

    for idx, frame_idx in enumerate(frame_idx_list, 1):
        if frame_idx >= frames_count:
            continue
        
        video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        success, frame = video_capture.read()
        if not success:
            break
        
        output_filename = os.path.join(output_folder, f"{str(idx).zfill(zfill_size)}.jpg")
        cv2.imwrite(output_filename, frame)
        
        pts_time = frame_idx / fps
        keyframes.append([idx, round(pts_time, 2), fps, frame_idx])
    
    video_capture.release()
    return keyframes 

In [15]:
for video_file in os.listdir(video_dir):
    if video_file.endswith('.mp4'):
        video_name = video_file.split('.')[0]
        video_path = os.path.join(video_dir, video_file)
        scenes_path = os.path.join(scenes_dir, f"{video_name}.json")
        keyframes_output_folder = os.path.join(keyframes_dir, video_name)
        map_keyframes_path = os.path.join(map_keyframes_dir, f"{video_name}.csv")

        with open(scenes_path, 'r') as f:
            scenes = json.load(f)

        frame_idx_list = get_frame_indices(scenes)
        keyframes = write_and_map_keyframes(video_path, frame_idx_list, keyframes_output_folder)

        df = pd.DataFrame(keyframes, columns=['n', 'pts_time', 'fps', 'frame_idx'])
        df.to_csv(map_keyframes_path, index=False)

